In [1]:
import polars as pl 
import requests

In [22]:
## read orig csv file, hardcode to this file
df = pl.read_csv("../data/historical_Data/PV-electricity_2024_01_01.csv", skip_lines=3, has_header=False, separator=";", new_columns=["date_time", "pv_electricity(kW)"], decimal_comma=True)
## convert date_time to datetime, other to numeric
df = df.with_columns(
    pl.col("date_time").str.to_datetime("%d.%m.%Y %H:%M")
)
df.head()

date_time,pv_electricity(kW)
datetime[μs],f64
2024-01-01 00:00:00,0.0
2024-01-01 00:15:00,0.0
2024-01-01 00:30:00,0.0
2024-01-01 00:45:00,0.0
2024-01-01 01:00:00,0.0


In [ ]:
## downsample to hourly data
df = df.sort("date_time")
df = df.group_by_dynamic("date_time", every="1h").agg(pl.col("pv_electricity(kW)").mean())

date_time,pv_electricity(kW)
datetime[μs],f64
2024-01-01 00:00:00,0.0
2024-01-01 01:00:00,0.0
2024-01-01 02:00:00,0.0
2024-01-01 03:00:00,0.0
2024-01-01 04:00:00,0.0
…,…
2025-07-07 20:00:00,3.3075
2025-07-07 21:00:00,0.586012
2025-07-07 22:00:00,0.0


In [53]:
## get sun position
from suncalc import get_position

lat= 53.5511
lon= 9.9937
date_time = df["date_time"].to_list()[0]
sun_pos = get_position(date_time, lng=lon, lat=lat)
print(sun_pos)

{'azimuth': np.float64(2.9575346136447305), 'altitude': np.float64(-1.0336211802887731)}


In [54]:
## calculate sun position for every timestamp
df = df.with_columns(
    pl.col("date_time").map_elements(lambda x: get_position(x, lng=lon, lat=lat)).alias("sun_position")
)


/tmp/ipykernel_14649/1301019818.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


In [55]:

df = df.with_columns(
    pl.col("sun_position").struct.unnest()
)
df = df.drop(["sun_position"])
df.head()

date_time,pv_electricity(kW),azimuth,altitude
datetime[μs],f64,f64,f64
2024-01-01 00:00:00,0.0,2.957535,-1.033621
2024-01-01 00:15:00,0.0,3.075297,-1.038464
2024-01-01 00:30:00,0.0,-3.089233,-1.038721
2024-01-01 00:45:00,0.0,-2.971291,-1.034387
2024-01-01 01:00:00,0.0,-2.855597,-1.025577
